In [95]:
# @author Shray Alag 
# @version 07.29.2020
# @mentors Abhi, Anshul
# @lab Kundaje Lab
#
# This notebook will taken in an HT-SELEX experiment and will split up the data inputs, 
# create a CNN, train the CNN, and evaluate it. This first cell is for establishing the path 
# directories, taking in the data, and filling up the kmer_Sequences (i.e. "ATTGCGG"), 
# count_Vals (i.e. "1403"), and probability_Vals (i.e. "0.0021591").




#import statements
from keras.callbacks import History, Callback, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os




#path directories
#To use this notebook simply change the experiment name and path directories to your liking. 
#Mostly just change the following variables and make sure your path directories are set up properly
#Also make sure to unzip everything
experiment_BASE_Name = "ZFP3_GC40NAGAGTT_Lysate_BatchAATA_Cycle"
num_Of_Kmers = 40


experiment_1_Name = experiment_BASE_Name + "1_R1." + str(num_Of_Kmers) + "mers.counts"
experiment_2_Name = experiment_BASE_Name + "2_R1." + str(num_Of_Kmers) + "mers.counts"
experiment_3_Name = experiment_BASE_Name + "3_R1." + str(num_Of_Kmers) + "mers.counts"



BASE_PATH_DIR = "/home/shray/HT-SELEX"
DATA_FOLDER_NAME = "HT-SELEX-" + str(num_Of_Kmers) + "mer-counts"
DATA_PATH_DIR = BASE_PATH_DIR + "/" + DATA_FOLDER_NAME


EXPERIMENT_DIR_1 = DATA_PATH_DIR + "/" + experiment_1_Name
EXPERIMENT_DIR_2 = DATA_PATH_DIR + "/" + experiment_2_Name
EXPERIMENT_DIR_3 = DATA_PATH_DIR + "/" + experiment_3_Name


print("Directories of this experiment are", EXPERIMENT_DIR_1, EXPERIMENT_DIR_2, EXPERIMENT_DIR_3)





#Global Vars
NUM_OF_SAMPLES_MUST_BE_IN_TRAINING = 2000
start_time = time.time()
global_time = time.time()
extra = np.zeros([num_Of_Kmers, 1])
counterFor3BasePairs = 0;
mask_value = -2;
keepInTraining = True
breakVal = 20000


#Overarching methods
def printOutArrays():
    # Prints out the shapes of the arrays
    print ("X_train shape: " + str(x_train.shape))
    print ("y_train shape: " + str(y_train.shape))
    print ("X_test shape: " + str(x_test.shape))
    print ("y_test shape: " + str(y_test.shape))
    print ("X_val shape: " + str(x_hidden.shape))
    print ("y_val shape: " + str(y_hidden.shape))
    
def dimensionsOfInputs():
    dim1 = len(kmer_Sequences_Train_ForSure)
    dim2 = len(kmer_Sequences_Train_ForSure[0])
    dim3 = len(kmer_Sequences_Train_ForSure[0][0])
    print("Dimensions of kmer_Sequences_Train_ForSure:", dim1, dim2, dim3)
    dim1 = len(kmer_Sequences)
    print("Dimensions of kmer_Sequences", dim1)
    dim1 = len(count_Vals_Train_ForSure)
    dim2 = len(count_Vals_Train_ForSure[0])
    print("Dimensions of count_Vals_Train_ForSure:", dim1, dim2)
    
    
    
    
# setting up the numpy arrays
# Following commented code is just for the user to see the data structures being used
kmer_Sequences_Train_ForSure = []
# kmer_Sequences_Train_ForSure = np.array(kmer_Sequences_Train_ForSure)
count_Vals_Train_ForSure = []
probability_Vals_Train_ForSure = []

kmer_Sequences = []
count_Vals = []
probability_Vals = []







#reading through and getting the training data + doing one hot encoding for the ones that are for sure in training
experimentReader1 = open(EXPERIMENT_DIR_1, "r")
experimentReader2 = open(EXPERIMENT_DIR_2, "r")
experimentReader3 = open(EXPERIMENT_DIR_3, "r")

experimentReader1.seek(0, os.SEEK_END)
size1 = experimentReader1.tell()

experimentReader2.seek(0, os.SEEK_END)
size2 = experimentReader2.tell()

experimentReader3.seek(0, os.SEEK_END)
size3 = experimentReader3.tell()

print("Sizes 1, 2, and 3 are", size1, size2, size3)
maxLen = max(size1, size2, size3)
print(maxLen)


experimentReader1 = open(EXPERIMENT_DIR_1, "r")
experimentReader2 = open(EXPERIMENT_DIR_2, "r")
experimentReader3 = open(EXPERIMENT_DIR_3, "r")

experimentReader1.readline() #don't want the first line with the headers (i.e. "k_mers", "counts") going into the data
experimentReader2.readline()
experimentReader3.readline()






for i in range(maxLen):

    
    if i < size1:
        line1 = experimentReader1.readline()
        elements = line1.split()
    
    
        if keepInTraining == True:
            count_Vals_Train_ForSure.append([elements[1], mask_value, mask_value])
            probability_Vals_Train_ForSure.append([elements[2], mask_value, mask_value])

            #One-hot encoding for the kmer Sequence
            stringTemp = list(elements[0].rstrip())  
            s = pd.Series(stringTemp)
            temp = pd.get_dummies(s)

            if temp.shape[1] == 3:
                counterFor3BasePairs = counterFor3BasePairs + 1
                temp = np.append(temp, extra, 1)

            if temp.shape[1] == 4 and temp.shape[0] == num_Of_Kmers:
                kmer_Sequences_Train_ForSure.append(np.array(temp))
    

        elif len(elements) >= 3:
            count_Vals.append([elements[1], mask_value, mask_value])
            probability_Vals.append([elements[2], mask_value, mask_value])
            kmer_Sequences.append(elements[0])
    
    
    
    
    if i < size2:
        line2 = experimentReader2.readline()
        elements = line2.split()
    
        if keepInTraining == True and len(elements) >= 3:
            count_Vals_Train_ForSure.append([mask_value, elements[1], mask_value])
            probability_Vals_Train_ForSure.append([mask_value, elements[2], mask_value])

            #One-hot encoding for the kmer Sequence
            stringTemp = list(elements[0].rstrip())  
            s = pd.Series(stringTemp)
            temp = pd.get_dummies(s)

            if temp.shape[1] == 3:
                counterFor3BasePairs = counterFor3BasePairs + 1
                temp = np.append(temp, extra, 1)

            if temp.shape[1] == 4 and temp.shape[0] == num_Of_Kmers:
                kmer_Sequences_Train_ForSure.append(np.array(temp))

        elif len(elements) >= 3:
            count_Vals.append([mask_value, elements[1], mask_value])
            probability_Vals.append([mask_value, elements[2], mask_value])

            kmer_Sequences.append(elements[0])
            
            
            
            
    if i < size3:
        line3 = experimentReader3.readline()
        elements = line3.split()
    
        if keepInTraining == True and len(elements) >= 3:
            count_Vals_Train_ForSure.append([mask_value, mask_value, elements[1]])
            probability_Vals_Train_ForSure.append([mask_value, mask_value, elements[2]])

            #One-hot encoding for the kmer Sequence
            stringTemp = list(elements[0].rstrip())  
            s = pd.Series(stringTemp)
            temp = pd.get_dummies(s)

            if temp.shape[1] == 3:
                counterFor3BasePairs = counterFor3BasePairs + 1
                temp = np.append(temp, extra, 1)

            if temp.shape[1] == 4 and temp.shape[0] == num_Of_Kmers:
                kmer_Sequences_Train_ForSure.append(np.array(temp))

        elif len(elements) >= 3:
            count_Vals.append([mask_value, mask_value, elements[1]])
            probability_Vals.append([mask_value, mask_value, elements[2]])
            kmer_Sequences.append(elements[0])

            
    if i >= NUM_OF_SAMPLES_MUST_BE_IN_TRAINING:
        keepInTraining = False
        
    if i >= breakVal:
        break
        

dimensionsOfInputs()
print(count_Vals_Train_ForSure[0])
print("--- %s seconds ---" % (time.time() - start_time))

Directories of this experiment are /home/shray/HT-SELEX/HT-SELEX-40mer-counts/ZFP3_GC40NAGAGTT_Lysate_BatchAATA_Cycle1_R1.40mers.counts /home/shray/HT-SELEX/HT-SELEX-40mer-counts/ZFP3_GC40NAGAGTT_Lysate_BatchAATA_Cycle2_R1.40mers.counts /home/shray/HT-SELEX/HT-SELEX-40mer-counts/ZFP3_GC40NAGAGTT_Lysate_BatchAATA_Cycle3_R1.40mers.counts
Sizes 1, 2, and 3 are 23822030 23312171 16338762
23822030
Dimensions of kmer_Sequences_Train_ForSure: 6003 40 4
Dimensions of kmer_Sequences 54000
Dimensions of count_Vals_Train_ForSure: 6003 3
['3.0', -2, -2]
--- 3.388583183288574 seconds ---


In [96]:
#expect this cell to take a few minutes

print(count_Vals_Train_ForSure[0])
print(count_Vals_Train_ForSure[0][0])

#import statements
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
print(pd.__version__)
start_time = time.time()






# Split the datasets into three parts - train, test, and hidden with a 70%, 15%, 15% split
# test datasets will be used to select the best model. 
# Hidden dataset will be used for final accuracy
# Makes sure top 2000 counts are in training set as it is important that the model learns these prominent sequences
x_train_temp, x_other, y_train, y_other = train_test_split(kmer_Sequences, count_Vals, test_size=0.3, random_state=42)
x_test_temp, x_hidden_temp, y_test, y_hidden = train_test_split(x_other, y_other, test_size=0.5, random_state=42)







#Doing one hot encoding for the rest of the data
x_train = np.zeros([len(x_train_temp), num_Of_Kmers, 4])
x_test = np.zeros([len(x_test_temp), num_Of_Kmers, 4])
x_hidden = np.zeros([len(x_hidden_temp), num_Of_Kmers, 4])


for i in range(len(x_train_temp)):
    stringTemp = list(x_train_temp[i])
    s = pd.Series(stringTemp)
    temp = pd.get_dummies(s)
    
    if temp.shape[1] == 3:
        counterFor3BasePairs = counterFor3BasePairs + 1
        temp = np.append(temp, extra, 1)
            
    if temp.shape[1] == 4:
        x_train[i] = temp
    
    if i < len(x_test_temp):
        stringTemp = list(x_test_temp[i])
        s = pd.Series(stringTemp)
        temp1 = pd.get_dummies(s)
        
        
        if temp1.shape[1] == 3:
            counterFor3BasePairs = counterFor3BasePairs + 1
            temp1 = np.append(temp1, extra, 1)
            
        if temp1.shape[1] == 4:
            x_test[i] = temp1 
    
    if i < len(x_hidden_temp):
        stringTemp = list(x_hidden_temp[i])
        s = pd.Series(stringTemp)
        temp2 = pd.get_dummies(s)
        
        if temp2.shape[1] == 3 and temp2.shape[0] == num_Of_Kmers:
            counterFor3BasePairs = counterFor3BasePairs + 1
            temp2 = np.append(temp2, extra, 1)
        
        if temp2.shape[1] == 4:
            x_hidden[i] = temp2
        
print("%s number of reads only had 3 base pairs" %counterFor3BasePairs)
    
    




    
    
    
#convert arrays to numpy
kmer_Sequences_Train_ForSure = np.array(kmer_Sequences_Train_ForSure)
count_Vals_Train_ForSure = np.array(count_Vals_Train_ForSure)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_hidden = np.array(y_hidden)

#making sure the one hot encoding worked 
print ("Kmer Sequences shape: " + str(kmer_Sequences_Train_ForSure.shape))
print ("Counts shape: " + str(count_Vals_Train_ForSure.shape))
printOutArrays()






#combines the training for sure samples and the training samples
x_train = np.concatenate((x_train, kmer_Sequences_Train_ForSure))
y_train = np.concatenate((y_train, count_Vals_Train_ForSure))




#print statements to make sure it worked
print ("Kmer Sequences shape: " + str(kmer_Sequences_Train_ForSure.shape))
print ("Counts shape: " + str(count_Vals_Train_ForSure.shape))
print("-----------")
printOutArrays()
print("--- %s seconds ---" % (time.time() - start_time))

['3.0', -2, -2]
3.0
1.1.0
51 number of reads only had 3 base pairs
Kmer Sequences shape: (6003, 40, 4)
Counts shape: (6003, 3)
X_train shape: (37800, 40, 4)
y_train shape: (37800, 3)
X_test shape: (8100, 40, 4)
y_test shape: (8100, 3)
X_val shape: (8100, 40, 4)
y_val shape: (8100, 3)
Kmer Sequences shape: (6003, 40, 4)
Counts shape: (6003, 3)
-----------
X_train shape: (43803, 40, 4)
y_train shape: (43803, 3)
X_test shape: (8100, 40, 4)
y_test shape: (8100, 3)
X_val shape: (8100, 40, 4)
y_val shape: (8100, 3)
--- 29.57035255432129 seconds ---


In [92]:
from numpy import unique
start_time = time.time()

temp1 = [3, 2, 2, 1, 5, 3]
temp1 = np.asarray(temp1)

def unique_by_first(a):
    tmp = a.reshape(a.shape[0], -1)
    b = np.ascontiguousarray(tmp).view(np.dtype((np.void, tmp.dtype.itemsize * tmp.shape[1])))
    _, idx, inverse = np.unique(b, return_index=True, return_inverse=True)
    return  a[idx].reshape(-1, *a.shape[1:]), inverse

uniqueElements_x_train, indexes_x_train = unique_by_first(temp1)


print(uniqueElements_x_train)
print(indexes_x_train)
print(uniqueElements_x_train[indexes_x_train])
print(len(indexes_x_train))
print("------")
print(len(uniqueElements_x_train))
print("len of x_train", len(x_train))

print("-------")
print("Percentage of kmer sequences crossover:", ((len(x_train) - len(uniqueElements_x_train))/len(x_train)) *100)


def removeDuplicatesWithY(indexesList, x_data, y_data):
    setOfIndexes = {}
    
    
print("--- %s seconds ---" % (time.time() - start_time))

[1 2 3 5]
[2 1 1 0 3 2]
[3 2 2 1 5 3]
6
------
4
len of x_train 43803
-------
Percentage of kmer sequences crossover: 99.99086820537406
--- 0.002225637435913086 seconds ---


In [93]:
from numpy import argsort
from numpy import unique
from numpy import split



start_time = time.time()

idx_sort = argsort(x_train)
sorted_records_array = x_train[idx_sort]
vals, idx_start, count = unique(sorted_records_array, return_counts=True,
                                return_index=True)

# sets of indices
res = split(idx_sort, idx_start[1:])
#filter them with respect to their size, keeping only items occurring more than once

vals = vals[count > 1]
res = filter(lambda x: x.size > 1, res)

print(res)
print(len(res))
print(vals)

print("--- %s seconds ---" % (time.time() - start_time))

TypeError: object of type 'filter' has no len()

In [72]:
seen = set()

sizeHidden = len(x_hidden)
sizeTest = len(x_test)

#precondition: x_train has to be bigger than hidden and test which should always be true
for i in range(len(x_train)):
    
    set.add(tuple(x_train[i]))
    
    if i < sizeHidden:
        set.add(x_hidden[i])
    
    if i < sizeTest:
        set.add(x_test[i])

TypeError: descriptor 'add' requires a 'set' object but received a 'tuple'

In [60]:
#This cell defines the CNN model

#import statements
from keras.layers import Dense, Conv1D, Flatten, Input, BatchNormalization
from keras.models import Sequential, Model
from keras import backend as K


#loss function
def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
    return K.binary_crossentropy(y_true * mask, y_pred * mask)


#defining the model
def CNNModel(input_shape):

    X_input = Input(input_shape)
    
    X = (X_input)
    
    
    #Layer1
    #change to conv2d
    X = Conv2D(16, (20, 4), name = 'conv1', strides = (1, 1), activation = 'relu')(X)
    X = BatchNormalization()(X)
    
    
    #Layer2
    X = Conv2D(16, (16, 4), name = 'conv2', strides = (1, 1), activation = 'relu')(X)
    X = BatchNormalization()(X)
    
  
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(3, activation='relu', name='fc')(X) #final predictions

    # Create model
    model = Model(inputs = X_input, outputs = X, name='CNNModel')
    
    return model

In [61]:
1000, 40, 4
mod = CNNModel([numOfSequences, kMerLength, oneHotEncoding])
mod.summary()

Model: "CNNModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 1000, 40, 4)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 981, 37, 16)       5136      
_________________________________________________________________
batch_normalization_16 (Batc (None, 981, 37, 16)       64        
_________________________________________________________________
conv2 (Conv2D)               (None, 966, 34, 32)       32800     
_________________________________________________________________
batch_normalization_17 (Batc (None, 966, 34, 32)       128       
_________________________________________________________________
conv3 (Conv2D)               (None, 959, 31, 64)       65600     
_________________________________________________________________
batch_normalization_18 (Batc (None, 959, 31, 64)       256

mod.summary()

In [ ]:
class Classifier(object):
    """
    Classifier interface.
    Args:
    Attributes:
        get_inputs (list): a list of input names.
            Derived from model_inputs unless implemented.
    """
    @property
    def get_inputs(self):
        return model_inputs[self.__class__.__name__]
    def __init__(self, **hyperparameters):
        pass
    def save(self, prefix):
        arch_fname = prefix + '.arch.json'
        weights_fname = prefix + '.weights.h5'
        open(arch_fname, 'w').write(self.model.to_json())
        self.model.save_weights(weights_fname, overwrite=True)
    def load_weights(self, filepath):
        self.model.load_weights(filepath)
    def get_placeholder_inputs(self):
        """Returns dictionary of named keras inputs"""
        return collections.OrderedDict(
            [(name, placeholder_dict[name])
             for name in self.get_inputs])
class SequenceClassifier(Classifier):
    def __init__(self,num_tasks=1,
                 num_filters=(15, 15, 15), conv_width=(15, 15, 15),
                 pool_width=35, dropout=0, batch_norm=False):
        assert len(num_filters) == len(conv_width)
        # # configure inputs
        ##Dictionary of data type to placeholder
        self.inputs = self.get_placeholder_inputs()
        # convolve sequence
        seq_preds = self.inputs["data/genome_data_dir"]
        for i, (nb_filter, nb_col) in enumerate(zip(num_filters, conv_width)):
            seq_preds = Conv1D(
                nb_filter, nb_col, kernel_initializer='he_normal')(seq_preds)
            if batch_norm:
                seq_preds = BatchNormalization()(seq_preds)
            seq_preds = Activation('relu')(seq_preds)
            if dropout > 0:
                seq_preds = Dropout(dropout)(seq_preds)
        # pool and fully connect
        seq_preds = AveragePooling1D((pool_width))(seq_preds)
        seq_preds = Flatten()(seq_preds)
        seq_preds = Dense(output_dim=num_tasks)(seq_preds)
        self.logits=seq_preds
        preds = Activation('sigmoid')(self.logits)
        self.model = Model(inputs=self.inputs.values(), outputs=preds)
    def get_logits(self):
    	return self.logits